In [ ]:
import json
import sys
import requests
from pyspark.sql import SparkSession
import os
from pyspark.sql.functions import lit
import socket
import time


In [ ]:
spark = SparkSession.builder \
    .appName("demoing") \
    .master("spark://spark-cluster-master-0.spark-cluster-headless.spark-cluster.svc.cluster.local:7077") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.1026") \
    .config("spark.executorEnv.JUPYTER_HOST", socket.gethostbyname(socket.gethostname())) \
    .config("spark.driver.host", socket.gethostbyname(socket.gethostname())) \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio.minio.svc.cluster.local:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "user") \
    .config("spark.hadoop.fs.s3a.secret.key", "password") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .enableHiveSupport() \
    .getOrCreate()

sc = spark.sparkContext

In [ ]:
start = time.time()

input_dir="s3a://demo/input/*" #2476
input_files = sc.textFile(input_dir)

words = input_files.flatMap(lambda line: line.split(" "))
word_counts = words.map(lambda word: (word, 1)).reduceByKey(lambda a,b: a+b)
sorted  = word_counts.sortBy(lambda pair: pair[1], ascending=True)
final = sorted.collect()
for (word, count) in final:
    print("%s: %i" % (word, count))
    
sorted.map(lambda x: f"{x[0], x[1]}").saveAsTextFile("s3a://demo/output/wordcount_jupyter")

end = time.time()
print(end - start)


In [ ]:
sc2 = sc._jsc.sc()
number_of_workers = len([executor.host() for executor in
                sc2.statusTracker().getExecutorInfos()]) - 1 

In [ ]:
number_of_workers